In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
import lightgbm as lgb
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
from datetime import datetime, timedelta
from functools import reduce
import numpy as np
import pandas as pd
import holidays
from sklearn.model_selection import train_test_split
import pandas as pd
import lightgbm as lgb

# Data Prep


In [3]:
from sklearn.model_selection import train_test_split
import pandas as pd
import lightgbm as lgb
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
from datetime import datetime, timedelta
from functools import reduce
import numpy as np
import pandas as pd
import holidays

#load data
df_ntc = pd.read_csv("../data/NTC.csv", parse_dates=['datetime'])
df_ntc.set_index('datetime', inplace=True)
df_ntc.index = df_ntc.index.tz_localize('Europe/Brussels', ambiguous='infer')
df_ntc.index = df_ntc.index.tz_convert('UTC')

df_act_loss = pd.read_csv("../data/Avtice-losses.csv", skiprows=1, parse_dates=['Zeitstempel'], index_col='Zeitstempel')
df_act_loss = df_act_loss.rename(columns={"kWh": "mwh", "Zeitstempel": "datetime"})
df_act_loss.index.name = 'datetime'
df_act_loss.index = df_act_loss.index.tz_localize('Europe/Brussels', ambiguous='infer')
df_act_loss.index = df_act_loss.index.tz_convert('UTC')

df_weather_forecast = pd.read_csv("../data/Forecast-renewable-generation.csv", parse_dates=['datetime'])
df_weather_forecast.set_index('datetime', inplace=True)
df_weather_forecast.index = df_weather_forecast.index.tz_localize('Europe/Brussels', ambiguous='infer')
df_weather_forecast.index = df_weather_forecast.index.tz_convert('UTC')

# df_weather_temp = pd.read_csv("../data/Forecast-temperature_new.csv", parse_dates=['datetime'])
# df_weather_temp.set_index('datetime', inplace=True)
# ambiguous_flags = [False] * len(df_weather_temp)  # Set all to False initially
# ambiguous_idx = df_weather_temp.index.get_loc(pd.Timestamp("2019-10-27 02:00:00"))
# ambiguous_flags[ambiguous_idx] = True  # Adjust as needed; True means it is DST
# df_weather_temp.index = df_weather_temp.index.tz_localize('Europe/Brussels', ambiguous=ambiguous_flags)
# df_weather_temp.index = df_weather_temp.index.tz_convert('UTC')


# You need to convert kWh to MWh by dividing it by 1000. As all other variables use MW.
df_act_loss['mwh'] = df_act_loss['mwh'].astype(float)
df_act_loss['mwh'] = df_act_loss['mwh'] / 1000.0

#downsample from 15 min to 1 hour
df_act_loss = df_act_loss.loc[~df_act_loss.index.duplicated(keep='first')]
df_act_loss = df_act_loss.resample('1H').sum()
df_act_loss = df_act_loss.interpolate()


#display
# display(df_ntc)
# display(df_act_loss)
# display(df_weather_forecast)
# # display(df_weather_temp)


###Merge###
# List of DataFrames
dfs = [df_ntc, df_weather_forecast, df_act_loss]

# Merge all DataFrames on 'datetime'
merged_df = reduce(lambda left, right: pd.merge(left, right, on='datetime', how='inner'), dfs)

# Function to determine the season
def determine_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Autumn'

# Function to determine if the date is a public holiday in Switzerland
def is_public_holiday(date):
    swiss_holidays = holidays.Switzerland(years=date.year)
    return 1 if date in swiss_holidays else 0

# Function to determine if the date is a special day in Switzerland (you can define your own special days)
def is_special_day(date):
    special_days = [pd.Timestamp(f'{date.year}-08-01'), pd.Timestamp(f'{date.year}-12-25'), pd.Timestamp(f'{date.year}-01-01')]
    return 1 if date in special_days else 0

# Add feature columns
def add_feature_columns(df):
    # Determine the season
    df['Season'] = df.index.month.map(determine_season)
    df_season = pd.get_dummies(df['Season'], prefix='', prefix_sep='')
    df = pd.concat([df, df_season], axis=1)
    df.drop('Season', axis=1, inplace=True)
    
    # Add month feature
    df['Month'] = df.index.month
    df = pd.get_dummies(df, columns=['Month'], prefix='Month')

    # Public Holidays and Special days
    df['Public_Holiday'] = df.index.to_series().map(is_public_holiday)
    df['Special_Day'] = df.index.to_series().map(is_special_day)

    # Add day of the week as a one-hot encoded feature
    df['Day_of_Week'] = df.index.dayofweek
    df = pd.get_dummies(df, columns=['Day_of_Week'], prefix='Day')

    return df

# Assuming merged_df is your DataFrame with datetime index
merged_df = add_feature_columns(merged_df)

#clean
merged_df.columns = merged_df.columns.str.replace('[":\[\]{}]', '', regex=True)
merged_df.columns = [col.replace(" ", "_") for col in merged_df.columns]
display(merged_df)

,CH_AT,CH_DE,CH_FR,CH_IT,AT_CH,DE_CH,FR_CH,IT_CH,solar_fore_de_MW,solar_fore_it_MW,...,Month_12,Public_Holiday,Special_Day,Day_0,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6
datetime,,,,,,,,,,,,,,,,,,,,,
2018-12-31 23:00:00+00:00,700.0,4000.0,1200.0,2513.0,1200.0,800.0,3000.0,1910.0,0.0,0.0,...,True,0,0,True,False,False,False,False,False,False
2019-01-01 00:00:00+00:00,700.0,4000.0,1200.0,2513.0,1200.0,800.0,3000.0,1910.0,0.0,0.0,...,False,1,0,False,True,False,False,False,False,False
2019-01-01 01:00:00+00:00,700.0,4000.0,1200.0,2513.0,1200.0,800.0,3000.0,1910.0,0.0,0.0,...,False,1,0,False,True,False,False,False,False,False
2019-01-01 02:00:00+00:00,700.0,4000.0,1200.0,2513.0,1200.0,800.0,3000.0,1910.0,0.0,0.0,...,False,1,0,False,True,False,False,False,False,False
2019-01-01 03:00:00+00:00,700.0,4000.0,1200.0,2513.0,1200.0,800.0,3000.0,1910.0,0.0,0.0,...,False,1,0,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31 18:00:00+00:00,1200.0,4000.0,1400.0,4069.0,1200.0,800.0,3200.0,1810.0,0.0,0.0,...,True,0,0,False,False,False,False,True,False,False
2021-12-31 19:00:00+00:00,1200.0,4000.0,1400.0,4069.0,1200.0,800.0,3200.0,1810.0,0.0,0.0,...,True,0,0,False,False,False,False,True,False,False
2021-12-31 20:00:00+00:00,1200.0,4000.0,1400.0,3953.0,1200.0,800.0,3200.0,1810.0,0.0,0.0,...,True,0,0,False,False,False,False,True,False,False


In [4]:
merged_df.columns

Index(['CH_AT', 'CH_DE', 'CH_FR', 'CH_IT', 'AT_CH', 'DE_CH', 'FR_CH', 'IT_CH',
       'solar_fore_de_MW', 'solar_fore_it_MW', 'wind_fore_de_MW',
       'wind_fore_it_MW', 'mwh', 'Autumn', 'Spring', 'Summer', 'Winter',
       'Month_1', 'Month_2', 'Month_3', 'Month_4', 'Month_5', 'Month_6',
       'Month_7', 'Month_8', 'Month_9', 'Month_10', 'Month_11', 'Month_12',
       'Public_Holiday', 'Special_Day', 'Day_0', 'Day_1', 'Day_2', 'Day_3',
       'Day_4', 'Day_5', 'Day_6'],
      dtype='object')

In [5]:
import warnings
warnings.filterwarnings("ignore")

# MODEL

In [6]:
df = merged_df
timeframe = 7 * 24

df['target'] = df['mwh']
feature_columns = ['mwh']


for col in feature_columns:
    for i in range(1, timeframe + 1):  # Loop over 1 day's worth of hours, adjust as needed
        lag_col_name = f'lag_{col}_{i}'  # Construct the lag column name
        df[lag_col_name] = df[col].shift(i)  # Initially create lag columns
        
        for idx, row in df.iterrows():
            lag_time = idx - pd.Timedelta(hours=i)
            
            # If the date portion of the lag time is the same as the current index, set it to NaN
            if lag_time.date() == idx.date():
                df.at[idx, lag_col_name] = np.nan

# Create moving averages for each feature
# for col in feature_columns:
#     # Existing moving averages
#     df[f'ma_{col}_6hr'] = df[col].rolling(window=6).mean()
#     df[f'ma_{col}_12hr'] = df[col].rolling(window=12).mean()
#     df[f'ma_{col}_1day'] = df[col].rolling(window=24).mean()
#     df[f'ma_{col}_7days'] = df[col].rolling(window=24*7).mean()

#     # Standard Deviation
#     df[f'std_{col}_6hr'] = df[col].rolling(window=6).std()
#     df[f'std_{col}_12hr'] = df[col].rolling(window=12).std()
#     df[f'std_{col}_1day'] = df[col].rolling(window=24).std()
#     df[f'std_{col}_7days'] = df[col].rolling(window=24*7).std()

#     # Skewness
#     df[f'skew_{col}_6hr'] = df[col].rolling(window=6).skew()
#     df[f'skew_{col}_12hr'] = df[col].rolling(window=12).skew()
#     df[f'skew_{col}_1day'] = df[col].rolling(window=24).skew()
#     df[f'skew_{col}_7days'] = df[col].rolling(window=24*7).skew()

#     # Kurtosis
#     df[f'kurt_{col}_6hr'] = df[col].rolling(window=6).kurt()
#     df[f'kurt_{col}_12hr'] = df[col].rolling(window=12).kurt()
#     df[f'kurt_{col}_1day'] = df[col].rolling(window=24).kurt()
#     df[f'kurt_{col}_7days'] = df[col].rolling(window=24*7).kurt()


df = df.iloc[timeframe:]

In [7]:
df

,CH_AT,CH_DE,CH_FR,CH_IT,AT_CH,DE_CH,FR_CH,IT_CH,solar_fore_de_MW,solar_fore_it_MW,...,lag_mwh_159,lag_mwh_160,lag_mwh_161,lag_mwh_162,lag_mwh_163,lag_mwh_164,lag_mwh_165,lag_mwh_166,lag_mwh_167,lag_mwh_168
datetime,,,,,,,,,,,,,,,,,,,,,
2019-01-07 23:00:00+00:00,700.0,4000.0,1200.0,1074.0,1200.0,800.0,3000.0,1910.0,0.0,0.0,...,195.088866,195.358274,179.847469,163.126507,142.718472,132.939009,135.187545,130.398025,133.772193,104.950331
2019-01-08 00:00:00+00:00,700.0,4000.0,1200.0,1074.0,1200.0,800.0,3000.0,1910.0,0.0,0.0,...,194.080276,195.088866,195.358274,179.847469,163.126507,142.718472,132.939009,135.187545,130.398025,133.772193
2019-01-08 01:00:00+00:00,700.0,4000.0,1200.0,1074.0,1200.0,800.0,3000.0,1910.0,0.0,0.0,...,179.687456,194.080276,195.088866,195.358274,179.847469,163.126507,142.718472,132.939009,135.187545,130.398025
2019-01-08 02:00:00+00:00,700.0,4000.0,1200.0,1097.0,1200.0,800.0,3000.0,1910.0,0.0,0.0,...,153.461735,179.687456,194.080276,195.088866,195.358274,179.847469,163.126507,142.718472,132.939009,135.187545
2019-01-08 03:00:00+00:00,700.0,4000.0,1200.0,1097.0,1200.0,800.0,3000.0,1910.0,0.0,0.0,...,146.970647,153.461735,179.687456,194.080276,195.088866,195.358274,179.847469,163.126507,142.718472,132.939009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31 18:00:00+00:00,1200.0,4000.0,1400.0,4069.0,1200.0,800.0,3200.0,1810.0,0.0,0.0,...,127.510133,115.127524,111.629891,103.791267,98.011972,99.350562,103.313038,103.859902,102.000354,97.671012
2021-12-31 19:00:00+00:00,1200.0,4000.0,1400.0,4069.0,1200.0,800.0,3200.0,1810.0,0.0,0.0,...,132.679227,127.510133,115.127524,111.629891,103.791267,98.011972,99.350562,103.313038,103.859902,102.000354
2021-12-31 20:00:00+00:00,1200.0,4000.0,1400.0,3953.0,1200.0,800.0,3200.0,1810.0,0.0,0.0,...,142.779857,132.679227,127.510133,115.127524,111.629891,103.791267,98.011972,99.350562,103.313038,103.859902


In [6]:
df.columns

Index(['CH_AT', 'CH_DE', 'CH_FR', 'CH_IT', 'AT_CH', 'DE_CH', 'FR_CH', 'IT_CH',
       'solar_fore_de_MW', 'solar_fore_it_MW',
       ...
       'lag_mwh_159', 'lag_mwh_160', 'lag_mwh_161', 'lag_mwh_162',
       'lag_mwh_163', 'lag_mwh_164', 'lag_mwh_165', 'lag_mwh_166',
       'lag_mwh_167', 'lag_mwh_168'],
      dtype='object', length=207)

In [7]:
df.fillna(-1, inplace=True)

In [8]:
from numba import cuda

# Initialize GPU
cuda.select_device(0)

# Reset GPU
cuda.close()

ModuleNotFoundError: No module named 'numba'

In [9]:
new_df = df.drop('mwh', axis=1)

# Sort the DataFrame by datetime
new_df.sort_index(inplace=True)

# Train-test split based on timestamp
train_size = int(len(new_df) * 0.8)
train, test = new_df.iloc[:train_size], new_df.iloc[train_size:]

# Features and target
X_train, y_train = train.drop('target', axis=1), train['target']
X_test, y_test = test.drop('target', axis=1), test['target']


In [10]:
# LightGBM parameters
params = {
    'objective': 'regression',
    'metric': 'mae',
    'boosting_type': 'gbdt',
    'device': 'gpu',
    'random_state': 42,
    'gpu_platform_id' : 0,
    'gpu_device_id': 0
}

# Create LightGBM data containers
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# Train the model
num_round = 50
bst = lgb.train(params, train_data, num_round, valid_sets=[test_data])

# Make predictions
y_pred = bst.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")

[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 44537
[LightGBM] [Info] Number of data points in the train set: 20908, number of used features: 204
[LightGBM] [Info] Using requested OpenCL platform 0 device 0
[LightGBM] [Info] Using GPU Device: NVIDIA A100 80GB PCIe, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 175 dense feature groups (3.51 MB) transferred to GPU in 0.134448 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 106.986450


KeyboardInterrupt: 

In [ ]:
import shap

# Initialize JavaScript for SHAP plots
shap.initjs()

# Create SHAP explainer
explainer = shap.Explainer(bst)

# Calculate SHAP values for test data
shap_values = explainer(X_test)

# Plot SHAP summary
shap.summary_plot(shap_values, X_test)


In [ ]:
# Initialize an empty list to store predictions
# predicted_values = []

# # Loop through each row in X_test
# for idx, _ in enumerate(X_test.index[:-1]):  # Loop until the second last row
#     input_data = X_test.iloc[[idx]]
    
#     # Make prediction
#     predicted_value = bst.predict(input_data)[0]
    
#     # Append the predicted value to the list
#     predicted_values.append(predicted_value)
    
#     # Update the next row's lagged feature with the predicted value
#     next_idx = idx + 1
#     X_test.at[X_test.index[next_idx], 'lag_mwh_1'] = predicted_value

# # Handle prediction for the last row separately as it doesn't affect any next row
# input_data = X_test.iloc[[-1]]
# predicted_value = bst.predict(input_data)[0]
# predicted_values.append(predicted_value)

# # Evaluate the model
# mse = mean_squared_error(y_test, predicted_values)
# rmse = np.sqrt(mse)
# mae = mean_absolute_error(y_test, predicted_values)

# print(f"RMSE: {rmse}")
# print(f"MAE: {mae}")


In [ ]:
# import numpy as np
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.metrics import mean_absolute_error, mean_squared_error

# # Normalize features
# scaler = MinMaxScaler(feature_range=(0, 1))
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# # Reshape input to be 3D [samples, timesteps, features]
# X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
# X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

# # Initialize the LSTM model
# model = Sequential()
# model.add(LSTM(10, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
# model.add(Dense(1))
# model.compile(optimizer='adam', loss='mean_absolute_error')

# # Train the model
# model.fit(X_train_reshaped, y_train, epochs=50, batch_size=72, validation_data=(X_test_reshaped, y_test), verbose=2, shuffle=False)

# # Make predictions
# y_pred = model.predict(X_test_reshaped)

# # Inverse scale predictions
# # y_pred_inv = scaler.inverse_transform(y_pred)

# # Evaluate the model
# mse = mean_squared_error(y_test, y_pred)
# rmse = np.sqrt(mse)
# mae = mean_absolute_error(y_test, y_pred)

# print(f"RMSE: {rmse}")
# print(f"MAE: {mae}")


In [ ]:
# y_pred_df = pd.DataFrame({'pred_loss': y_pred}, index=X_test.index)

# # 2. Reshape the DataFrame
# # Group by date and create a dictionary
# pred_dict = {}
# for date, group in y_pred_df.groupby(y_pred_df.index.date):
#     daily_preds = group['pred_loss'].values
#     if len(daily_preds) == 24:  # Only take days that have full 24 hours data
#         pred_dict[date] = daily_preds

# # 3. Save each day's observations in a dictionary as specified
# output_dict = {}
# for date, hourly_preds in pred_dict.items():
#     base_datetime = datetime.combine(date, datetime.min.time())
#     output_dict[str(date)] = {
#         (base_datetime + timedelta(hours=hour)).strftime("%Y-%m-%d %H:%M:%S"): pred
#         for hour, pred in enumerate(hourly_preds)
#     }

# # Print the result (or you can save it to a file)
# print(output_dict)

In [ ]:
X_train[["lag_mwh_1", "lag_mwh_2", "lag_mwh_3", "lag_mwh_4", "lag_mwh_5"]].tail(24)